In [1]:
###IMPORTS###
#to run a server, config allows us to change the general look, allows use of CSS
from pywebio import start_server, config
#allows input fields to remain on page after submission (Makes fields persistent)
from pywebio.pin import *
#allows use of input commands
from pywebio.input import *
#allows use of output commands
from pywebio.output import *

import Database as db


In [2]:
def main_menu():
    db.open_cookbook()
    options = db.getRecipeList()
    put_text(options)
    print(options)

In [3]:
start_server(main_menu, port = 8080, debug = True ) 

Running on all addresses.
Use http://10.57.193.193:8080/ to access the application


RuntimeError: This event loop is already running

['Pizza']
